In [1]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import json
import glob
import os

In [2]:
paths = glob.glob("../exp/bidir_features/*.json")
data = [json.load(open(path)) for path in paths]

In [30]:
queries = pd.DataFrame.from_records([{
    **algo,
    'graph': { 'europe': 'DIMACs Europe', 'osm_ger': 'OSM Germany', 'osm_europe': 'OSM Europe' }[[x for x in run['args'][1].split('/') if x != ''][-1]],
    'experiment': exp['experiment'],
    'factor': exp['factor'],
    'probability': exp.get('probability', 1),
    'speed': exp.get('speed', 0),
    'potential': run['potential'],
    'num_queue_pops': algo['num_queue_pops'] + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo['num_queue_pushs'] + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo['num_relaxed_arcs'] + algo.get('core_search', {}).get('num_relaxed_arcs', 0),
    }
    for run in data for exp in run['experiments'] for algo in exp['algo_runs']])

queries['choose_direction_strategy'] = queries['choose_direction_strategy'].fillna('unidir')
queries['bidir_pot'] = queries['bidir_pot'].fillna('unidir')
queries['improved_pruning'] = queries['improved_pruning'].fillna('unidir')

In [4]:
pd.set_option('display.max_rows', 500)

In [32]:
queries = queries.loc[lambda x: x['graph'] == 'OSM Germany']

In [38]:
queries.groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'choose_direction_strategy', 'improved_pruning'])[['running_time_ms', 'num_queue_pushs']].mean().unstack(0)

running_time_ms  \
potential                                                                                                                                                  ALT   
experiment                   factor probability algo                                      bidir_pot choose_direction_strategy improved_pruning                   
probabilistic_scale_by_speed 1.25   0.00        Bidrectional Dijkstra Query               Average   alternating               False                     562.86   
                                                                                                                              True                      563.81   
                                                                                                    min_key                   False                     584.80   
                                                                                                                              True                      590.12   
                                                                                          Symmetric alternating               False                   1,777.17   
                                                                                                                              True                      573.14   
                                                                                                    min_key                   False                   1,761.24   
                                                                                                                              True                      605.35   
                                                Dijkstra Query                            unidir    unidir                    unidir                    711.28   
                                                Virtual Topocore Bidirectional Core Query Average   alternating               False                     158.72   
                                                                                                                              True                      160.36   
                                                                                                    min_key                   False                     165.85   
                                                                                                                              True                      167.22   
                                                                                          Symmetric alternating               False                     592.90   
                                                                                                                              True                      176.69   
                                                                                                    min_key                   False                     582.03   
                                                                                                                              True                      186.60   
                                                Virtual Topocore Component Query          unidir    unidir                    unidir                    216.35   
                                    1.00        Bidrectional Dijkstra Query               Average   alternating               False                      63.61   
                                                                                                                              True                       64.38   
                                                                                                    min_key                   False                      88.06   
                                                                                                                              True                       89.21   
                                                                                          Sym

In [40]:
queries.loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'improved_pruning'])[['running_time_ms', 'num_queue_pushs']].mean().unstack(0) \
    .loc[('weight_scale', 1.05, 1.00)] \
    .droplevel(2)

running_time_ms        \
potential                                                       ALT    CH   
algo                                      bidir_pot                         
Bidrectional Dijkstra Query               Average            130.99 56.46   
                                          Symmetric          270.95 53.85   
Dijkstra Query                            unidir             317.37 43.76   
Virtual Topocore Bidirectional Core Query Average             34.53 17.85   
                                          Symmetric           75.87 19.65   
Virtual Topocore Component Query          unidir              95.95 16.16   

                                                                     \
potential                                           Oracle     Zero   
algo                                      bidir_pot                   
Bidrectional Dijkstra Query               Average    33.90 1,555.31   
                                          Symmetric  33.75 1,545.56   
Dijkstra Query                            unidir     31.63 2,261.84   
Virtual Topocore Bidirectional Core Query Average     8.97   395.43   
                                          Symmetric  10.37   396.72   
Virtual Topocore Component Query          unidir     10.74   612.21   

                                                    num_queue_pushs  \
potential                                                       ALT   
algo                                      bidir_pot                   
Bidrectional Dijkstra Query               Average        307,354.53   
                                          Symmetric      649,905.62   
Dijkstra Query                            unidir         859,615.38   
Virtual Topocore Bidirectional Core Query Average         60,110.07   
                                          Symmetric      129,586.95   
Virtual Topocore Component Query          unidir         179,126.41   

                                                                           \
potential                                                   CH     Oracle   
algo                                      bidir_pot                         
Bidrectional Dijkstra Query               Average   116,325.20 116,325.20   
                                          Symmetric 108,874.76 108,874.76   
Dijkstra Query                            unidir    126,889.23 126,889.23   
Virtual Topocore Bidirectional Core Query Average    21,891.49  21,891.49   
                                          Symmetric  19,164.68  19,164.68   
Virtual Topocore Component Query          unidir     24,915.00  24,915.00   

                                                                  
potential                                                   Zero  
algo                                      bidir_pot               
Bidrectional Dijkstra Query               Average   4,797,134.33  
                                          Symmetric 4,794,507.37  
Dijkstra Query                            unidir    8,422,394.67  
Virtual Topocore Bidirectional Core Query Average     975,982.68  
                                          Symmetric   967,819.16  
Virtual Topocore Component Query          unidir    1,750,108.15

### Effectiveness new pruning

In [41]:
queries.loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'improved_pruning'])[['running_time_ms', 'num_queue_pushs']].mean().unstack(0) \
    .loc[('weight_scale', 1.05, 1.00)]

running_time_ms  \
potential                                                                        ALT   
algo                                      bidir_pot improved_pruning                   
Bidrectional Dijkstra Query               Average   False                     130.99   
                                                    True                      132.76   
                                          Symmetric False                     797.74   
                                                    True                      270.95   
Virtual Topocore Bidirectional Core Query Average   False                      34.53   
                                                    True                       35.24   
                                          Symmetric False                     244.80   
                                                    True                       75.87   

                                                                             \
potential                                                                CH   
algo                                      bidir_pot improved_pruning          
Bidrectional Dijkstra Query               Average   False             56.46   
                                                    True              57.97   
                                          Symmetric False            112.55   
                                                    True              53.85   
Virtual Topocore Bidirectional Core Query Average   False             17.85   
                                                    True              18.01   
                                          Symmetric False             38.05   
                                                    True              19.65   

                                                                             \
potential                                                            Oracle   
algo                                      bidir_pot improved_pruning          
Bidrectional Dijkstra Query               Average   False             33.90   
                                                    True              34.57   
                                          Symmetric False             81.50   
                                                    True              33.75   
Virtual Topocore Bidirectional Core Query Average   False              8.97   
                                                    True               9.18   
                                          Symmetric False             24.12   
                                                    True              10.37   

                                                                               \
potential                                                                Zero   
algo                                      bidir_pot improved_pruning            
Bidrectional Dijkstra Query               Average   False            1,555.31   
                                                    True             1,550.62   
                                          Symmetric False            6,137.12   
                                                    True             1,545.56   
Virtual Topocore Bidirectional Core Query Average   False              395.43   
                                                    True               399.60   
                                          Symmetric False            1,641.51   
                                                    True               396.72   

                                                                     num_queue_pushs  \
potential                                                                        ALT   
algo                                      bidir_pot improved_pruning                   
Bidrectional Dijkstra Query               Average   False                 307,354.53   
                                                    True                  305,887.93   
                            

### Min key is Quatsch

In [42]:
queries.loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot', 'choose_direction_strategy'])[['running_time_ms', 'num_queue_pushs']].mean().unstack(0) \
    .loc[('probabilistic_scale_by_speed', 1.50)]

running_time_ms                \
potential                                                   ALT     CH Oracle   
probability bidir_pot choose_direction_strategy                                 
0.00        Average   alternating                        281.72 368.40 211.48   
                      min_key                            287.57 379.83 215.08   
            Symmetric alternating                        287.84 373.11 225.56   
                      min_key                            294.64 394.81 251.02   
1.00        Average   alternating                         19.06   8.97   4.31   
                      min_key                             31.21  16.39   8.05   
            Symmetric alternating                         37.11   9.88   5.06   
                      min_key                             55.28  19.04   9.95   

                                                       num_queue_pushs  \
potential                                         Zero             ALT   
probability bidir_pot choose_direction_strategy                          
0.00        Average   alternating               403.00      435,385.09   
                      min_key                   429.89      456,898.67   
            Symmetric alternating               400.60      443,585.97   
                      min_key                   422.61      473,216.77   
1.00        Average   alternating               384.59       33,441.39   
                      min_key                   417.73       52,656.98   
            Symmetric alternating               389.14       57,967.07   
                      min_key                   416.76       87,446.87   

                                                                       \
potential                                               CH     Oracle   
probability bidir_pot choose_direction_strategy                         
0.00        Average   alternating               403,958.89 403,958.89   
                      min_key                   418,515.15 418,515.15   
            Symmetric alternating               393,025.96 393,025.96   
                      min_key                   416,010.83 416,010.83   
1.00        Average   alternating                11,169.04  11,169.04   
                      min_key                    19,587.80  19,587.80   
            Symmetric alternating                 9,665.48   9,665.48   
                      min_key                    18,122.65  18,122.65   

                                                              
potential                                               Zero  
probability bidir_pot choose_direction_strategy               
0.00        Average   alternating               1,057,218.68  
                      min_key                   1,118,974.37  
            Symmetric alternating               1,050,733.25  
                      min_key                   1,112,446.50  
1.00        Average   alternating                 893,603.40  
                      min_key                     956,629.08  
            Symmetric alternating                 884,751.97  
                      min_key                     947,262.51

### Bidir Effectiveness

In [45]:
queries.loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot'])[['running_time_ms', 'num_queue_pushs']].mean().unstack(0)

running_time_ms  \
potential                                                             ALT   
experiment                   factor probability bidir_pot                   
probabilistic_scale_by_speed 1.25   0.00        Average            158.72   
                                                Symmetric          176.69   
                                                unidir             216.35   
                                    1.00        Average             17.21   
                                                Symmetric           37.35   
                                                unidir              75.84   
                             1.50   0.00        Average            281.72   
                                                Symmetric          287.84   
                                                unidir             334.79   
                                    1.00        Average             19.06   
                                                Symmetric           37.11   
                                                unidir              93.30   
weight_scale                 1.00   1.00        Average             15.07   
                                                Symmetric           42.85   
                                                unidir              43.37   
                             1.05   1.00        Average             34.53   
                                                Symmetric           75.87   
                                                unidir              95.95   
                             1.10   1.00        Average             63.15   
                                                Symmetric           99.61   
                                                unidir             130.45   

                                                                         \
potential                                                     CH Oracle   
experiment                   factor probability bidir_pot                 
probabilistic_scale_by_speed 1.25   0.00        Average   184.53 103.21   
                                                Symmetric 193.09 112.28   
                                                unidir    170.32 123.20   
                                    1.00        Average     4.50   2.01   
                                                Symmetric   5.07   2.35   
                                                unidir      7.18   4.61   
                             1.50   0.00        Average   368.40 211.48   
                                                Symmetric 373.11 225.56   
                                                unidir    318.50 228.56   
                                    1.00        Average     8.97   4.31   
                                                Symmetric   9.88   5.06   
                                                unidir     18.96  12.61   
weight_scale                 1.00   1.00        Average     0.83   0.19   
                                                Symmetric   0.71   0.20   
                                                unidir      0.48   0.17   
                             1.05   1.00        Average    17.85   8.97   
                                                Symmetric  19.65  10.37   
                                                unidir     16.16  10.74   
                             1.10   1.00        Average    52.29  27.97   
                                                Symmetric  57.01  31.18   
                                                unidir     56.70  39.33   

                                                                  \
potential                                                   Zero   
experiment                   factor probability bidir_pot          
probabilistic_scale_by_speed 1.25   0.00        Average   398.44   
                                                Symmetric 397.80   
                                                unidir    574.75   
